In [1]:
import mlrun


project_name = "widsdb2"
project_dir = "conf"

wids_prj, artifact_path = mlrun.set_environment('http://mlrun-api:8080', 
                                                project=project_name, user_project=False)

widsdb2_proj = mlrun.projects.load_project(project_dir,  clone=True)






In [2]:
%%nuclio cmd -c
pip install lightgbm

In [3]:
# nuclio: start-code

In [4]:
from sklearn import linear_model
from sklearn import datasets
from sklearn.model_selection import train_test_split
from cloudpickle import dumps
import pandas as pd

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from mlrun.mlutils.data import get_sample
from mlrun.mlutils.plots import eval_model_v2
from scipy.stats import kurtosis, iqr, skew, gmean, hmean, mode, normaltest, shapiro, ks_2samp
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from logging import getLogger, Formatter, StreamHandler, FileHandler, INFO, ERROR
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import os, gc, sys, time, random, math
from contextlib import contextmanager
from matplotlib import pyplot as plt
from IPython.display import display
from scipy import stats, special
from sklearn import set_config
from functools import partial
import lightgbm as lgb
import seaborn as sns
import pandas as pd
import typing as tp
import numpy as np
import warnings


warnings.simplefilter('ignore')

In [5]:
def train_wids(context: MLClientCtx,
               dataset: DataItem,
               label_column: str = "labels"):
    
    raw, labels, header = get_sample(dataset, sample=-1, label=label_column)
    print(raw.shape)
    # Basic scikit-learn Iris data-set SVM model
    X_train, X_test, y_train, y_test = train_test_split(
        raw, labels, test_size=0.2, random_state=42)
    
    context.log_dataset('train_set', 
                        df=pd.concat([X_train, y_train.to_frame()], axis=1),
                        format='csv', index=False, 
                        artifact_path=context.artifact_subpath('data'))

    context.log_dataset('test_set', 
                        df=pd.concat([X_test, y_test.to_frame()], axis=1),
                        format='csv', index=False, 
                        labels={"data-type": "held-out"},
                        artifact_path=context.artifact_subpath('data'))
   

    lparams = {}

    evals = {}
    evals['lgb'] = {}
    lparams = {}

    seeds = [0, 1]
    nfold = [5, 5]
    stratified=[False, False]
    shuffle=[True, True]
    n_estimators = 30000
    early_stopping_rounds = 300
    verbose_eval = 2
    learning_rate = 0.01
    reg_alpha = [0.4, 1]
    reg_lambda = [0.7, 1]
    subsample = [0.45, 1]
    colsample_bytree = [0.3, 0.225]
    max_depth = -1
    verbose = 4
    n_jobs = 4
    # Create callback class for retrieving trained model.
    #extraction_cb = ModelExtractionCallback()
    #callbacks = [
    #    extraction_cb,
    #]
    
    lparams = dict(boosting_type='gbdt',
               objective='binary',
               metric='auc',
               learning_rate= learning_rate,
               num_leaves= 200,
               max_bin=500,
               min_child_weight= 0.035,
               subsample= subsample[0],
               colsample_bytree= colsample_bytree[0],
               min_data_in_leaf= 150,
               max_depth= max_depth,
               bagging_seed= seeds[0],
               reg_alpha= reg_alpha[0],
               reg_lambda= reg_lambda[0],
               verbose= verbose,
               seed= seeds[0],
               n_jobs= n_jobs,)

    evals_result = {}
    test_preds = np.zeros(len(X_test))    
    dtrain = lgb.Dataset(X_train, y_train)
    dtest = lgb.Dataset(X_test, label=y_test)
    
    #dtest = X_test.copy()
    testlen = X_test.shape[0]
    
    seed = seeds[0]
    print(f'Training Model with SEED : {seed}')
    i=0
    model = lgb.train(lparams, dtrain, 100, valid_sets=dtest, early_stopping_rounds=5, evals_result=evals_result)
    model.save_model('model.txt', num_iteration=model.best_iteration)
    
   
     # Evaluate model results and get the evaluation metrics
    #eval_metrics = eval_model_v2(context, X_test, y_test, model)
   
    
    #score = model.best_score['valid_0']['auc']
    pred = model.predict(X_test)
    

    context.log_model("model",
                      body=dumps(model),
                      model_file="model.pkl",
                      metrics=context.results,
                      labels={"class": "lightgbm"})
    

In [6]:
# nuclio: end-code

In [7]:
train_wids_func = mlrun.code_to_function(name='train_wids',
                                         handler='train_wids',
                                         kind='job',
                                         image='mlrun/ml-models')

In [8]:
#train_wids_func.save()


In [9]:
fn = f'db://widsdb2/train-wids'
f = mlrun.import_function(fn)
f

In [10]:
widsdb2_proj.set_function(f)

In [11]:
widsdb2_proj.save()

In [12]:
artifact_path

dfa = mlrun.get_dataitem('store://widsdb2/prep-trdata_prep_train_enc:latest')

dfa

'v3io:///projects/widsdb2/artifacts/train_enc.csv'

In [13]:
## The preprocessed train data is located here 


dataset = f'store://widsdb2/prep-trdata_prep_train_enc:latest'
df = mlrun.get_dataitem(dataset).as_df()
df.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,...,d1_resprate_div_sysbp_min,d1_lactate_min_div_diasbp_min,d1_heartrate_min_div_d1_sysbp_min,d1_hco3_div,d1_resprate_times_resprate,left_average_spo2,total_chronic,total_cancer_immuno,has_complicator,apache_3j
0,68.0,22.732803,0,2,1,180.3,4,1,92,0,...,0.136986,0.027027,0.986301,1.266667,340.0,91.333333,0,0,0,7
1,77.0,27.421875,0,2,0,160.0,4,1,90,0,...,0.179104,0.112903,1.074627,1.038462,384.0,90.000000,0,0,0,6
2,25.0,31.952749,0,2,0,172.7,3,0,93,0,...,0.076190,0.055834,0.647619,1.088020,168.0,95.666667,0,0,0,8
3,81.0,22.635548,1,2,0,165.1,8,2,92,0,...,0.083333,0.055834,1.095238,1.088020,161.0,98.333333,0,0,0,4
4,19.0,29.054816,0,2,1,188.0,15,0,91,0,...,0.133333,0.055834,0.500000,1.088020,288.0,98.666667,0,0,0,8


In [14]:
#dfx = df.dropna()
#raw = raw.iloc[:sample, :]
df.isna().sum()       

age                    0
bmi                    0
elective_surgery       0
ethnicity              0
gender                 0
                      ..
left_average_spo2      0
total_chronic          0
total_cancer_immuno    0
has_complicator        0
apache_3j              0
Length: 525, dtype: int64

In [15]:
df['diabetes_mellitus']

0         1
1         1
2         0
3         0
4         0
         ..
130152    0
130153    0
130154    1
130155    0
130156    0
Name: diabetes_mellitus, Length: 130157, dtype: int64

In [16]:
df.columns

Index(['age', 'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source', 'icu_admit_source', 'icu_id', 'icu_stay_type',
       ...
       'd1_resprate_div_sysbp_min', 'd1_lactate_min_div_diasbp_min',
       'd1_heartrate_min_div_d1_sysbp_min', 'd1_hco3_div',
       'd1_resprate_times_resprate', 'left_average_spo2', 'total_chronic',
       'total_cancer_immuno', 'has_complicator', 'apache_3j'],
      dtype='object', length=525)

In [17]:
dfa = df
dfa['diabetes_mellitus']

0         1
1         1
2         0
3         0
4         0
         ..
130152    0
130153    0
130154    1
130155    0
130156    0
Name: diabetes_mellitus, Length: 130157, dtype: int64

In [18]:
train_wids_func.apply(mlrun.platforms.auto_mount())
train_wids_func.run(inputs={'dataset': dataset},
                                params={'label_column': 'diabetes_mellitus'})

> 2021-07-01 12:36:11,349 [info] starting run train-wids-train_wids uid=dc7f93e30aa34c87a1475219ab6c1e06 DB=http://mlrun-api:8080
> 2021-07-01 12:36:11,750 [info] Job is running in the background, pod: train-wids-train-wids-p9dhp
(130156, 524)
Training Model with SEED : 0
[LightGBM] [Info] Number of positive: 22459, number of negative: 81665
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.851008
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.522800
[LightGBM] [Debug] init for col-wise cost 0.229498 seconds, init for row-wise cost 0.370799 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.391584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 66462
[LightGBM] [Info] Number of data points in the train set: 104124, number of used features: 518
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215695 -> initscore=-1.290934
[LightGBM] [Info] Start 

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
widsdb2,...ab6c1e06,0,Jul 01 12:36:18,completed,train-wids-train_wids,v3io_user=aruna_lankakind=jobowner=aruna_lankahost=train-wids-train-wids-p9dhp,dataset,label_column=diabetes_mellitus,,train_settest_setmodel


to track results use .show() or .logs() or in CLI: 
!mlrun get run dc7f93e30aa34c87a1475219ab6c1e06 --project widsdb2 , !mlrun logs dc7f93e30aa34c87a1475219ab6c1e06 --project widsdb2
> 2021-07-01 12:37:51,823 [info] run executed, status=completed


In [21]:
train_run = train_wids_func.run(inputs={'dataset': dataset},
                                params={'label_column': 'diabetes_mellitus'},
                               local=True)

> 2021-07-01 12:38:22,697 [info] starting run train-wids-train_wids uid=0cb6650d1ed34619b173db506d3ae472 DB=http://mlrun-api:8080
(130156, 524)
Training Model with SEED : 0
[1]	valid_0's auc: 0.752827
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.769704
[3]	valid_0's auc: 0.811719
[4]	valid_0's auc: 0.824077
[5]	valid_0's auc: 0.824225
[6]	valid_0's auc: 0.823175
[7]	valid_0's auc: 0.826803
[8]	valid_0's auc: 0.829315
[9]	valid_0's auc: 0.831111
[10]	valid_0's auc: 0.833628
[11]	valid_0's auc: 0.835518
[12]	valid_0's auc: 0.836519
[13]	valid_0's auc: 0.836474
[14]	valid_0's auc: 0.837285
[15]	valid_0's auc: 0.83829
[16]	valid_0's auc: 0.838227
[17]	valid_0's auc: 0.838745
[18]	valid_0's auc: 0.839503
[19]	valid_0's auc: 0.839855
[20]	valid_0's auc: 0.839679
[21]	valid_0's auc: 0.839848
[22]	valid_0's auc: 0.839709
[23]	valid_0's auc: 0.839978
[24]	valid_0's auc: 0.840041
[25]	valid_0's auc: 0.839973
[26]	valid_0's auc: 0.840239
[27]	valid_0's auc: 0.8

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
widsdb2,...6d3ae472,0,Jul 01 12:38:22,completed,train-wids-train_wids,v3io_user=aruna_lankakind=owner=aruna_lankahost=jupyter-svc-68f96fcc4c-wjtq2,dataset,label_column=diabetes_mellitus,,train_settest_setmodel


to track results use .show() or .logs() or in CLI: 
!mlrun get run 0cb6650d1ed34619b173db506d3ae472 --project widsdb2 , !mlrun logs 0cb6650d1ed34619b173db506d3ae472 --project widsdb2
> 2021-07-01 12:39:47,120 [info] run executed, status=completed


In [22]:
print (train_run.outputs['model'])



store://artifacts/widsdb2/train-wids-train_wids_model:0cb6650d1ed34619b173db506d3ae472


In [23]:
print (train_run.outputs['test_set'])

store://artifacts/widsdb2/train-wids-train_wids_test_set:0cb6650d1ed34619b173db506d3ae472


In [24]:
from mlrun.platforms import auto_mount
describe = mlrun.import_function('hub://describe').apply(auto_mount())
test_classifier = mlrun.import_function('hub://test-classifier').apply(auto_mount())

In [25]:
widsdb2_proj.set_function(describe)
widsdb2_proj.set_function(test_classifier)



In [26]:
widsdb2_proj.save()

In [27]:
describe_run = describe.run(params={'label_column': 'diabetes_mellitus2'},
                            inputs={"table":
                                    train_run.outputs['test_set']})

> 2021-07-01 12:40:09,454 [info] starting run describe-summarize uid=313f1dfa5dc54f7e831bd5f4a1e48e7b DB=http://mlrun-api:8080
> 2021-07-01 12:40:09,775 [info] Job is running in the background, pod: describe-summarize-6mv6s


KeyboardInterrupt: 

In [73]:
from IPython.display import display, HTML
# Display the `histograms` artifact
display(HTML(mlrun.get_dataitem(describe_run.outputs['histograms']).get().decode('utf-8')))



KeyError: 'histograms'

In [ ]:
# Display the `imbalance` artifact
display(HTML(mlrun.get_dataitem(describe_run.outputs['imbalance']).get().decode('utf-8')))

In [ ]:
# Display the `correlation` artifact
display(HTML(mlrun.get_dataitem(describe_run.outputs['correlation']).get().decode('utf-8')))

In [ ]:
test = mlrun.import_function('hub://test_classifier').apply(auto_mount())# Display the model accuracy
print(f'Test Accuracy: {test_run.outputs["accuracy"]}')

# Display HTML output artifacts
display(HTML(mlrun.get_dataitem(test_run.outputs['confusion-matrix']).get().decode('utf-8')))
display(HTML(mlrun.get_dataitem(test_run.outputs['roc-multiclass']).get().decode('utf-8')))

In [15]:
#pred, model, res = lgbtrain(dfa, 'diabetes_mellitus')

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
#tn, fp, fn, tp = confusion_matrix(pred, [1, 1, 1, 0]).ravel()